<a href="https://colab.research.google.com/github/Ci-re/cirec-01-llm-falcon/blob/main/restore_punctuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install deepmultilingualpunctuation

In [ ]:
from deepmultilingualpunctuation import PunctuationModel

model = PunctuationModel()
text = "Hello good morning james how are you doing"
result = model.restore_punctuation(text)
print(result)

In [ ]:
! pip install dask[complete]

In [ ]:
import dask.bag as db
import glob
import json



dd_data = db.read_text("*.json").map(json.loads)
dd_data

In [ ]:
import dask.dataframe as dd
import pandas as pd
from deepmultilingualpunctuation import PunctuationModel

def restore_punctuation_safe(text, model):
    if isinstance(text, str):
        return model.restore_punctuation(text)
    else:
        return ""

files_list = glob.glob("*.json")
for file in files_list:
  dat = pd.read_json(file)
  dat = dd.from_pandas(dat, npartitions=4)
  dat = dat[["minister", "channel_name", "title", "description", "transcript"]]

  model = PunctuationModel()

  meta = dat.head(0)

  dat = dat[dat['transcript'].str.len() > 10]

  ddat = dat.map_partitions(
      lambda df: df.assign(transcript=df['transcript'].apply(restore_punctuation_safe, args=(model,))),
      meta=meta
  )

  ddat = ddat.compute()
  with open(f"{file}_new.json", "w") as f_in:
    json.dump(ddat.to_dict(orient="records"), f_in)

In [ ]:
ddat